# Lab 3: Text clustering

Generate and interpret document and term clusters from hotel reviews

Objectives:

- work with sklearn components
- construct document-term and term-document matrices
- apply dimensionality reduction
- perform cluster analysis on document and term collections
- interpret results

In [2]:
import re

import nltk
import numpy as np
import pandas as pd
from cytoolz import *

In [3]:
pd.set_option("display.max_colwidth", None)

In [4]:
df = pd.read_pickle("/data/hotels_id.pkl")
df.head()['offering_id']


0      93338
1      93338
2    1762573
3    1762573
4    1762573
Name: offering_id, dtype: int64

In [5]:
df.head(3)

,title,text,overall,value,service,cleanliness,offering_id
0,"“Truly is ""Jewel of the Upper Wets Side""”","Stayed in a king suite for 11 nights and yes it cots us a bit but we were happy with the standard of room, the location and the friendliness of the staff. Our room was on the 20th floor overlooking Broadway and the madhouse of the Fairway Market. Room was quite with no noise evident from the hallway or adjoining rooms. It was great to be able to open windows when we craved fresh rather than heated air. The beds, including the fold out sofa bed, were comfortable and the rooms were cleaned well. Wi-fi access worked like a dream with only one connectivity issue on our first night and this was promptly responded to with a call from the service provider to ensure that all was well. The location close to the 72nd Street subway station is great and the complimentary umbrellas on the drizzly days were greatly appreciated. It is fabulous to have the kitchen with cooking facilities and the access to a whole range of fresh foods directly across the road at Fairway.\nThis is the second time that members of the party have stayed at the Beacon and it will certainly be our hotel of choice for future visits.",5.0,5.0,5.0,5.0,93338
1,“My home away from home!”,"On every visit to NYC, the Hotel Beacon is the place we love to stay. So conveniently located to Central Park, Lincoln Center and great local restaurants. The rooms are lovely - beds so comfortable, a great little kitchen and new wizz bang coffee maker. The staff are so accommodating and just love walking across the street to the Fairway supermarket with every imaginable goodies to eat (if you choose not to go out for every meal!)",5.0,5.0,5.0,5.0,93338
2,“Great Stay”,"This is a great property in Midtown. We two different rooms different rooms during our stay. The first room was in the North tower, which was quite inconvenient. You have to go through the conference area to get to the north elevators. \nThe second room was the Andaz Suite. It was nicely appointed room, but the best part about it was the bathroom. From the foot soaking bowl to the bath products, everything about the bathroom was awesome!\nLemon poppy-seed pancakes are must haves at the restaurant. One of the best pancakes ever.",4.0,4.0,4.0,5.0,1762573


In [6]:
ny = df.query('offering_id==93338')

In [7]:
len(ny)

1192

In [8]:
ny.head()

,title,text,overall,value,service,cleanliness,offering_id
0,"“Truly is ""Jewel of the Upper Wets Side""”","Stayed in a king suite for 11 nights and yes it cots us a bit but we were happy with the standard of room, the location and the friendliness of the staff. Our room was on the 20th floor overlooking Broadway and the madhouse of the Fairway Market. Room was quite with no noise evident from the hallway or adjoining rooms. It was great to be able to open windows when we craved fresh rather than heated air. The beds, including the fold out sofa bed, were comfortable and the rooms were cleaned well. Wi-fi access worked like a dream with only one connectivity issue on our first night and this was promptly responded to with a call from the service provider to ensure that all was well. The location close to the 72nd Street subway station is great and the complimentary umbrellas on the drizzly days were greatly appreciated. It is fabulous to have the kitchen with cooking facilities and the access to a whole range of fresh foods directly across the road at Fairway.\nThis is the second time that members of the party have stayed at the Beacon and it will certainly be our hotel of choice for future visits.",5.0,5.0,5.0,5.0,93338
1,“My home away from home!”,"On every visit to NYC, the Hotel Beacon is the place we love to stay. So conveniently located to Central Park, Lincoln Center and great local restaurants. The rooms are lovely - beds so comfortable, a great little kitchen and new wizz bang coffee maker. The staff are so accommodating and just love walking across the street to the Fairway supermarket with every imaginable goodies to eat (if you choose not to go out for every meal!)",5.0,5.0,5.0,5.0,93338
17,“Excellent location”,"Loved the hotel. Great location - only 2 blocks from 72nd St subway. We had a 1 bedroom suite on the 24th floor and had a wonderful view of the Hudson River and the upper West Side. Lovely subtle green decor and room large for NYC standards. It was great having the living area with the lounge that converted into a double bed for our 2 children (12 and 14). Kitchenette was handy to eat breakfast in. We did cook a few evening meals but were limited with cutlery! The Fairway market across the road from the hotel was fabulous. It had an amazing variety of produce as well as ready cooked meals. Having the laundry on the 7th floor was extremely handy. US$1.75 per load and you could get change from the front reception. The staff and front doormen were all very friendly and helpful. Next to the lift is an info desk where you could get a map of NYC. We used this map for the subway, streets and main attractions as these were all on it. Good bottle shop next door to hotel as well.",5.0,5.0,5.0,5.0,93338
18,“All-round fantastic NYC hotel”,"Our first stay on the upper west side and can't say enough good things about the Hotel Beacon. We stayed here based on reviews of family friendly hotels. I'm not sure exactly how they are more family friendly than any other hotel except for the suites and kitchenettes for budget-conscious families. There weren't any special programs for kids, but we did see other families there. For NYC, a nice sized Queen suite gave us everything we needed. Didn't use the kitchen for our overnight stay but definitely would in the future. Rooms were comfortable (ok... a little tight at the foot of the bed but not like some complaints described), the amenities were really appreciated, e.g., free wifi, Keurig coffee service in room, desk area, energy efficient rooms (you put your room card in a device that controls the electricity to save energy while you're out). Lovely bathrooms (although they need a hand towel holder), but really comfortable. We are a family of 3 so the kiddo slept on the pull-out sofa. Room was in good condition-- it appears that shade of green must be the theme, but its nice. The bellman held our luggage after checkout to allow us to stay in the city and not have to worry about our 

## Construct a document-term matrix

See the [Sklearn user manual](https://scikit-learn.org/stable/modules/feature_extraction.html#text-feature-extraction) for functions for constructing dtm's. The most useful for this lab are [`CountVectorizer`](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html)  and [`TfidfVectorizer`](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html).  Both have lots of options you can try adjusting, e.g.,:

- min_df: ignore words that occur in fewer than `min_df` documents (either count or percentage)
- max_df: ignore words that occur in more than `max_df` documents (either count or percentage)
- stop_words: if set to `"english"`, removes common English function words
- analyzer, tokenizer, preprocessor, lowercase, strip_accents: controls sklearn's built-in tokenizer. If you want to use your own tokenizer (like the multi-word one we made last week), set `analyzer=identity` and give the vectorizer pre-tokenized texts.

`TfidfVectorizer` adds a few additional options:

- norm: set to `l2` (the default) to normalize doc vectors to unit length or `None` for no normalization
- use_idf: include idf term
- smooth_idf: add one to document frequencies
- sublinear_tf: replace tf term with 1+log(tf)

And see docs for many other options!

In [9]:
from sklearn.feature_extraction.text import CountVectorizer

In [10]:
tf = CountVectorizer(min_df=3, max_df=0.8)
#tfidf for normalised vec
dtm = tf.fit_transform(ny["text"])

In [11]:
dtm

<1192x3030 sparse matrix of type '<class 'numpy.int64'>'
	with 106638 stored elements in Compressed Sparse Row format>

In [12]:
from sklearn.decomposition import TruncatedSVD

## Apply dimensionality reduction

The original formulation of Latent Semantic Indexing used SVD for dimensionality reduction. The sklearn function [`TruncatedSVD`](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.TruncatedSVD.html)  is an efficient implentation of SVD for large sparse matrices (such as dtm's). The argument `n_components` sets the number of dimensions (i.e., columns) in the reduced matrix.

You can experiment with other dimensionality reduction techniques, as described in the manual sections on [Matrix factorization](https://scikit-learn.org/stable/modules/decomposition.html#lsa) and [Manifold](https://scikit-learn.org/stable/modules/manifold.html#t-sne). Or, you can skip this step and use your dtm directly as input to the clusterer



In [13]:
lsi = TruncatedSVD(n_components=28, random_state=760)
dtm_lsi = lsi.fit_transform(dtm)

In [14]:
dtm_lsi.shape # no 0's

(1192, 28)

## Perform cluster analysis

Sklearn has implementations of most current clusterining algoroths. See the [clustering chapter](https://scikit-learn.org/stable/modules/clustering.html#clustering) for more discussion. 

The [k-means](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html) algorithm is a particular simple one. It requires one argument: `n_clusters` sets the number of clusters that k-means will find. K-means is a non-deterministic algorithm, meaning each time you run it you may get a different answer. If you want your results to be reproducible, set the `random_state` argument.




In [15]:
from sklearn.cluster import KMeans

In [16]:
kmeans = KMeans(n_clusters=30, random_state=670,n_init="auto")# cluster 15
cluster = kmeans.fit_predict(dtm_lsi)# d points assigned to clusters
# 

## Interpret results

Now the hard part: what does all this mean? There are two ways of evaluating a clustering solution, quantitatively and qualitatively.

In [17]:
from sklearn import metrics

A simple quantitative metric: how many documents are in each cluster?

In [18]:
nltk.FreqDist(cluster).most_common()# 

[(5, 204),
 (11, 126),
 (8, 104),
 (0, 92),
 (22, 81),
 (9, 69),
 (26, 69),
 (18, 67),
 (21, 58),
 (28, 49),
 (2, 47),
 (25, 34),
 (6, 30),
 (12, 28),
 (1, 24),
 (7, 19),
 (13, 18),
 (19, 16),
 (16, 14),
 (14, 12),
 (20, 9),
 (23, 7),
 (24, 5),
 (4, 3),
 (3, 2),
 (10, 1),
 (17, 1),
 (15, 1),
 (27, 1),
 (29, 1)]

More complex metrics take into account how diffuse the clusters are and how well separated they are. Three potentially useful metrics are:

- [Calinski Harabasz_score](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.calinski_harabasz_score.html#sklearn.metrics.calinski_harabasz_score) [higher = better]
- [Davies Bouldin score](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.davies_bouldin_score.html#sklearn.metrics.davies_bouldin_score) [lower = better]
- [Silhouette_score](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.silhouette_score.html#sklearn.metrics.silhouette_score) [higher = better]

These are mostly useful for comparing different clusterings of the same data. For example, we could use these to find a good value for *k*, the number of clusters






In [19]:
for k in range(2, 20):
    _kmeans = KMeans(n_clusters=k, random_state=858, n_init="auto")
    _cluster = _kmeans.fit_predict(dtm_lsi)
    print(
        f"{k:2}  "
        f"{metrics.calinski_harabasz_score(dtm_lsi, _cluster):7.3f}   "
        f"{metrics.davies_bouldin_score(dtm_lsi, _cluster):7.3f}   "
        f"{metrics.silhouette_score(dtm_lsi, _cluster):7.3f}  "
    )

 2  404.236     1.453     0.466  
 3  294.006     1.913     0.240  
 4  226.876     2.284     0.167  
 5  196.693     2.233     0.157  
 6  171.937     2.374     0.145  
 7  151.976     2.381     0.132  
 8  138.989     2.421     0.100  
 9  122.499     2.597     0.085  
10  114.194     2.669     0.069  
11  106.564     2.529     0.076  
12  102.730     2.472     0.078  
13   95.477     2.515     0.081  
14   90.961     2.472     0.054  
15   86.839     2.450     0.037  
16   83.168     2.460     0.042  
17   79.455     2.471     0.046  
18   76.191     2.542     0.036  
19   73.164     2.550     0.030  


Qualitative evaluation means reading the data and trying to figure it out. We could look at all of the reviews in some of the smaller clusters:

In [20]:
ny[cluster == 6]["text"].head()

280                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          We were a group of 4 adults and 3 children so we booked 2 1 bedroom suites which were perfect. The bedrooms were spacious and the lounge area had a lot of room to move aro

Or, since we're using *k* means, we could sort them in order of proximity to the cluster centroid.

In [21]:
C = 6
distances = metrics.pairwise.euclidean_distances(
    kmeans.cluster_centers_[C, np.newaxis, :], dtm_lsi[cluster == C, :] 
)
ny[cluster == C].iloc[np.argsort(distances)[0]]["text"].head(5)

19516                                                                                                                                                                                                 A good friend told me about Hotel Beacon, and after reading all the other reviews, we booked for Dec. 21 to Dec. 27. From the moment we arrived until the bill was taken care of, we could not have had a more pleasant experience. The lobby was elegantly decorated for Christmas. \nThe room was very comfortable (but admittedly, the beds were somewhat hard; nevertheless, we slept just fine), and having the kitchenette made life so much easier. Everything was kept sparkling clean throughout our stay-plenty of strong, hot water for baths...A booklet in the room gives suggestions on where to eat, and the places that we tried matched what was said. Good markets and stores all around, and having the subway all of a minute away was great. People who stayed at the hotel were friendly-maybe we all felt l

## Term clustering

Document clustering is not usually very helpful, unless the documents are very short and/or naturally fall into a small number of distinct classes. Term clustering, on the other hand, does sometimes give interesting results. 

To cluster terms, all we need to do is transpose the document-term matrix to form the term-document matrix (tdm). After that, the rest of the process is the same, except that instead of relating documents by the terms they share, we're relating terms by the documents they co-occur in.



In [22]:
tf = CountVectorizer(min_df=2, max_df=0.5)
tdm = tf.fit_transform(ny["text"]).T

lsi = TruncatedSVD(n_components=6, random_state=858)
tdm_lsi = lsi.fit_transform(tdm)

kmeans = KMeans(15, random_state=858, n_init="auto")
cluster = kmeans.fit_predict(tdm_lsi)

In [23]:
nltk.FreqDist(cluster).most_common()

[(0, 3294),
 (13, 440),
 (7, 148),
 (2, 42),
 (8, 38),
 (11, 23),
 (12, 14),
 (10, 12),
 (14, 10),
 (4, 8),
 (1, 3),
 (9, 2),
 (5, 1),
 (6, 1),
 (3, 1)]

In [24]:
vocab = tf.get_feature_names_out()

In [25]:
vocab[cluster == 11]

array(['about', 'again', 'also', 'an', 'area', 'bed', 'bedroom', 'by',
       'door', 'floor', 'here', 'kitchenette', 'next', 'nice', 'night',
       'only', 'other', 'really', 'small', 'suite', 'two', 'us', 'well'],
      dtype=object)

In [26]:
C = 11
distances = metrics.pairwise.euclidean_distances(
    kmeans.cluster_centers_[C, np.newaxis, :], tdm_lsi[cluster == C, :]
)
vocab[cluster == C][np.argsort(distances)[0]]
# ranking acc to dis from centroid

array(['by', 'again', 'floor', 'really', 'small', 'about', 'kitchenette',
       'here', 'next', 'other', 'two', 'door', 'area', 'bedroom', 'night',
       'nice', 'bed', 'an', 'well', 'only', 'suite', 'also', 'us'],
      dtype=object)

In [27]:
vocab[cluster == 8]

array(['72nd', 'around', 'away', 'bathroom', 'beds', 'blocks',
       'breakfast', 'broadway', 'city', 'close', 'comfortable', 'easy',
       'etc', 'excellent', 'fairway', 'few', 'food', 'friendly',
       'helpful', 'kitchen', 'little', 'market', 'minutes', 'nights',
       'perfect', 'recommend', 'restaurants', 'right', 'road', 'safe',
       'side', 'spacious', 'square', 'station', 'supermarket', 'upper',
       'walking', 'west'], dtype=object)

In [28]:
C = 8
distances = metrics.pairwise.euclidean_distances(
    kmeans.cluster_centers_[C, np.newaxis, :], tdm_lsi[cluster == C, :]
)
vocab[cluster == C][np.argsort(distances)[0]]
# ranking acc to dis from centroid

array(['breakfast', 'square', 'kitchen', 'upper', 'easy', 'nights',
       'restaurants', 'recommend', 'excellent', 'food', 'minutes',
       'walking', 'west', 'city', 'little', 'market', 'side', 'spacious',
       '72nd', 'station', 'road', 'beds', 'safe', 'broadway', 'close',
       'perfect', 'supermarket', 'few', 'fairway', 'friendly',
       'comfortable', 'around', 'helpful', 'bathroom', 'etc', 'right',
       'away', 'blocks'], dtype=object)

## Results

Once you're done experimenting with document and term clustering, write up a paragraph on your results? What methods did you try? What worked and what didn't? Did you manage to find any useful information about hotels from this?

In [ ]:
Methods I tried-
I have done  analysis on the hotel with offering ID-93338.
I have changed some parameters like dimensionality, cluster number 
I ignored the words which occur less than 3 times and constructed a dtm
I tried to perform clustering with DBSCAN but I got an error and I’m unable to figure it out why.
Instead of using TfidfVectorizer, I used CountVectorizer and got the following results


Paragraph on results and useful information inferred from results-
From  the first 5 rows of ny by the code-ny.head(), we can say that it is a hotel in NYC.
After performing a dimensionality reduction on a term document matrix using TruncatedSVD to identify latent topics or concepts in a set of documents and formed clusters with K-Means algorithm,I Observed terms in cluster-11 and Cluster-8

Cluster-11
The terms in Cluster 11 are not much significant. There are terms like- by, here, next, other and two , which doesn’t give much information about the document.

Cluster-8
The terms in cluster 8 are 
array(['breakfast', 'square', 'kitchen', 'upper', 'easy', 'nights',
       'restaurants', 'recommend', 'excellent', 'food', 'minutes',
       'walking', 'west', 'city', 'little', 'market', 'side', 'spacious',
       '72nd', 'station', 'road', 'beds', 'safe', 'broadway', 'close',
       'perfect', 'supermarket', 'few', 'fairway', 'friendly',
       'comfortable', 'around', 'helpful', 'bathroom', 'etc', 'right',
       'away', 'blocks'], dtype=object)

Which has significant words like -easy, recommend, excellent, spacious, safe, perfect, friendly, comfortable, helpful-which indicate that the hotel with ID-93338 has positive reviews
And the terms like-breakfast,kitchen, restaurants, food, supermarket-which indicates that this cluster is formed on the theme- Food 

To conclude, TfidfVectorizer is better than CountVectorizer. As,
CountVectorizer does not take into account the fact that some words may occur very frequently in some documents, but less frequently in others.
Whereas, TfidfVectorizer-takes into account  how important a word is to a document in a collection of documents. Words that occur frequently in a document but also in many other documents across the collection are down-weighted
#TfidfVectorizer is a more sophisticated approach that can provide better results in some text analysis tasks by taking 
#into account the  relative importance of words in a document.

What worked and what didn’t-

CountVectorizer worked well. DBSCAN didn’t work well.